<a href="https://colab.research.google.com/github/twilight2001/FEBRL-fork-v0.4.2/blob/master/blazingsql_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get Started with Blazing SQL

In this notebook, we will walk through the process for getting BlazingSQL and cuDF running. We will then demonstrate a basic ETL process and query a csv. 

Please click the 'Open in Playground Mode Option ' to run this demo on your own. 

Feel free to add your own code or modify ours as needed.


Background Info:
[BlazingSQL](http://blazingdb.com/#/)
[RAPIDS AI/cuDF](https://rapids.ai/about.html)


# Setup

## Environment Sanity Check 

Once you are in Playground mode, click the Runtime dropdown at the top of the page, then Change Runtime Type and confirm the instance type has GPU selected for 'Hardware Accelerator'.

Check the output of '!nvidia-smi' to make sure you've been allocated a Tesla T4.


In [1]:
!nvidia-smi

Tue Aug 27 12:33:20 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Sometimes Google Colab will assign a Tesla K80 (which we do not support), so the script below helps confirm you have a T4 GPU attached to your runtime.

In [2]:
import pynvml


pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)

if device_name != b'Tesla T4':
  raise Exception("""
    Unfortunately this instance does not have a T4 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.

    If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print('Woo! You got the right kind of GPU!')

Woo! You got the right kind of GPU!


## Install BlazingSQL + cuDF
This will take roughly a minute and a half.

cuDF is the RAPIDS AI GPU DataFrame library with a Pandas-like API. Learn more about cuDF [here](https://github.com/rapidsai/cudf).

In [2]:
%%writefile bsql-colab.sh

#!/bin/bash


set -eu

wget -nc https://github.com/rapidsai/notebooks-extended/raw/master/utils/env-check.py
echo "Checking for GPU type:"
python env-check.py

if [ ! -f Miniconda3-4.5.4-Linux-x86_64.sh ]; then
    echo "Removing conflicting packages, will replace with RAPIDS compatible versions"
    # remove existing xgboost and dask installs
    pip uninstall -y xgboost dask distributed

    # intall miniconda
    echo "Installing conda"
    wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
    chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
    bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
    
    echo "Installing RAPIDS packages"
    echo "Please standby, this will take a few minutes..."
    # install RAPIDS packages
    conda install -y --prefix /usr/local \
      -c rapidsai/label/xgboost -c rapidsai -c nvidia -c conda-forge \
      python=3.6 cudatoolkit=10.0 \
      cudf=0.9.* cuml=0.9.* cugraph=0.9.* gcsfs pynvml \
      dask-cudf=0.9.* \
      rapidsai/label/xgboost::xgboost=>0.9
      
    echo "Copying shared object files to /usr/lib"
    # copy .so files to /usr/lib, where Colab's Python looks for libs
    cp /usr/local/lib/libcudf.so /usr/lib/libcudf.so
    cp /usr/local/lib/librmm.so /usr/lib/librmm.so
    cp /usr/local/lib/libxgboost.so /usr/lib/libxgboost.so
    cp /usr/local/lib/libnccl.so /usr/lib/libnccl.so
    conda install -y --prefix /usr/local -c rapidsai -c nvidia -c conda-forge -c felipeblazing/label/cuda10.0 python=3.6 cudatoolkit=10.0 blazingsql-ral blazingsql-orchestrator blazingsql-calcite blazingsql-python
    pip install flatbuffers
fi
echo ""
echo "*********************************************"
echo "Your Google Colab instance is RAPIDS ready!"
echo "*********************************************"



echo ""
echo "*********************************************"
echo "Your Google Colab instance is BlazingSQL ready!"
echo "*********************************************"


Overwriting bsql-colab.sh


In [3]:
%%time
!bash bsql-colab.sh

--2019-08-27 12:38:02--  https://github.com/rapidsai/notebooks-extended/raw/master/utils/env-check.py
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/rapidsai/notebooks-contrib/raw/master/utils/env-check.py [following]
--2019-08-27 12:38:02--  https://github.com/rapidsai/notebooks-contrib/raw/master/utils/env-check.py
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/env-check.py [following]
--2019-08-27 12:38:03--  https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/env-check.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.13

# You are ready to go with BlazingSQL!
Nice Job! Now lets see how it works.

# Import packages and create Blazing Context
You can think of the BlazingContext much like a Spark Context. This is where information such as FileSystems you have registered, Tables you have created will be stored.

If you have issues running this cell, restart runtime and try running it again.


In [0]:
import sys, os, time

sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

#Standup the BlazingSQL Services - We are working on removing the need to call these functions and just initializing them in BlazingContext
import subprocess
subprocess.Popen(['blazingsql-orchestrator', '9100', '8889', '127.0.0.1', '8890'],stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
subprocess.Popen(['java', '-jar', '/usr/local/lib/blazingsql-algebra.jar', '-p', '8890'])
import pyblazing.apiv2.context as cont
cont.runRal()
time.sleep(1) #Wait for service to start.

In [5]:
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

connection established


# Read CSV
First we uploaded a CSV file through the Google Colab interface. 

Then we use cuDF to read the CSV file. This gives us a GPU DataFrame (GDF).

To learn more about the GDF and how it enables end to end workloads on rapids, read [this](https://blog.blazingdb.com/blazingsql-part-1-the-gpu-dataframe-gdf-and-cudf-in-rapids-ai-96ec15102240).

In [6]:
#Download the test CSV
!wget 'https://s3.amazonaws.com/blazingsql-colab/Music.csv'

--2019-08-27 12:46:09--  https://s3.amazonaws.com/blazingsql-colab/Music.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.165.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.165.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10473 (10K) [text/csv]
Saving to: ‘Music.csv’

Music.csv           100%[===================>]  10.23K  --.-KB/s    in 0s      

2019-08-27 12:46:10 (78.2 MB/s) - ‘Music.csv’ saved [10473/10473]



In [0]:
gdf = cudf.read_csv('Music.csv')

# Create a Table

Now we just need to create a table. Apache Parquet is a great, open source and distributed file format built for systems like HDFS. Apache Parquet files also have metadata which self describes the schema, making import a cinch!

In [0]:
bc.create_table('music', gdf)

# Query a Table
That's it! Now you can write a SQL query and the data will get processed on the GPU with BlazingSQL, and the output will be a GPU DataFrame (GDF) inside RAPIDS!

In [10]:
#Query
result = bc.sql('SELECT * FROM main.music').get()

#Get GDF
result_gdf = result.columns

#Print GDF
print(result_gdf)

                                  ARTIST RATING  ...          LOCATION FESTIVAL_SET
0                            Arcade Fire   10.0  ...         Las Vegas          1.0
1                                Justice   10.0  ...         Las Vegas          1.0
2               Florence and The Machine   10.0  ...         Las Vegas          1.0
3                                 Odesza   10.0  ...             Indio          1.0
4                               Bon Iver   10.0  ...             Indio          1.0
5            LA Philharmonic + Sigur Ros   10.0  ...                LA          0.0
6                              Sigur Ros   10.0  ...             Malmo          0.0
7                            Arcade Fire   10.0  ...             Indio          1.0
8                                 Escort    9.0  ...     San Francisco          0.0
9                                Phoenix    9.0  ...          Berkeley          0.0
10                              Jamie XX    9.0  ...  Golden Gate Park      

And... thats it! You are now live with BlazingSQL. Check out our [docs](https://docs.blazingdb.com) to get fancy as well as to learn more about how BlazingSQL works with the rest of [RAPIDS AI](https://rapids.ai/). 

In [12]:
%%writefile bsql-colab.sh

#!/bin/bash


set -eu

wget -nc https://github.com/rapidsai/notebooks-extended/raw/master/utils/env-check.py
echo "Checking for GPU type:"
python env-check.py

if [ ! -f Miniconda3-4.5.4-Linux-x86_64.sh ]; then
    echo "Removing conflicting packages, will replace with RAPIDS compatible versions"
    # remove existing xgboost and dask installs
    pip uninstall -y xgboost dask distributed

    # intall miniconda
    echo "Installing conda"
    wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
    chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
    bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
    
    echo "Installing RAPIDS packages"
    echo "Please standby, this will take a few minutes..."
    # install RAPIDS packages
    conda install -y --prefix /usr/local \
      -c rapidsai/label/xgboost -c rapidsai -c nvidia -c conda-forge \
      python=3.6 cudatoolkit=10.0 \
      cudf=0.9.* cuml=0.9.* cugraph=0.9.* gcsfs pynvml \
      dask-cudf=0.9.* \
      rapidsai/label/xgboost::xgboost=>0.9
      
    echo "Copying shared object files to /usr/lib"
    # copy .so files to /usr/lib, where Colab's Python looks for libs
    cp /usr/local/lib/libcudf.so /usr/lib/libcudf.so
    cp /usr/local/lib/librmm.so /usr/lib/librmm.so
    cp /usr/local/lib/libxgboost.so /usr/lib/libxgboost.so
    cp /usr/local/lib/libnccl.so /usr/lib/libnccl.so
    conda install -y --prefix /usr/local -c rapidsai -c nvidia -c conda-forge -c felipeblazing/label/cuda10.0 python=3.6 cudatoolkit=10.0 blazingsql-ral blazingsql-orchestrator blazingsql-calcite blazingsql-python
    pip install flatbuffers
fi
echo ""
echo "*********************************************"
echo "Your Google Colab instance is RAPIDS ready!"
echo "*********************************************"



echo ""
echo "*********************************************"
echo "Your Google Colab instance is BlazingSQL ready!"
echo "*********************************************"


Overwriting bsql-colab.sh


In [13]:
--2019-08-23 23:02:04--  https://github.com/rapidsai/notebooks-extended/raw/master/utils/env-check.py
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/rapidsai/notebooks-contrib/raw/master/utils/env-check.py [following]
--2019-08-23 23:02:04--  https://github.com/rapidsai/notebooks-contrib/raw/master/utils/env-check.py
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/env-check.py [following]
--2019-08-23 23:02:04--  https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/env-check.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 783 [text/plain]
Saving to: ‘env-check.py’

env-check.py        100%[===================>]     783  --.-KB/s    in 0s      

2019-08-23 23:02:04 (184 MB/s) - ‘env-check.py’ saved [783/783]

Checking for GPU type:
*********************************************
Woo! Your instance has the right kind of GPU!
*********************************************

Removing conflicting packages, will replace with RAPIDS compatible versions
Uninstalling xgboost-0.90:
  Successfully uninstalled xgboost-0.90
Uninstalling dask-1.1.5:
  Successfully uninstalled dask-1.1.5
Uninstalling distributed-1.25.3:
  Successfully uninstalled distributed-1.25.3
Installing conda
--2019-08-23 23:02:08--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M  88.6MB/s    in 0.6s    

2019-08-23 23:02:08 (88.6 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11.0-py36h372c433_1 ...
installing: cffi-1.11.5-py36h9745a5d_0 ...
installing: setuptools-39.2.0-py36_0 ...
installing: cryptography-2.2.2-py36h14c3975_0 ...
installing: wheel-0.31.1-py36_0 ...
installing: pip-10.0.1-py36_0 ...
installing: pyopenssl-18.0.0-py36_0 ...
installing: urllib3-1.22-py36hbe7ace6_0 ...
installing: requests-2.18.4-py36he2e5f8d_1 ...
installing: conda-4.5.4-py36_0 ...
installation finished.
WARNING:
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local
Installing RAPIDS packages
Please standby, this will take a few minutes...


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.4
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base conda


sqlite-3.29.0        |  2.0 MB | : 100% 1.0/1 [00:00<00:00,  3.04it/s]               
cuml-0.9.1           |  5.9 MB | : 100% 1.0/1 [00:01<00:00,  1.49s/it]               
toolz-0.10.0         |   46 KB | : 100% 1.0/1 [00:00<00:00, 16.08it/s]
c-ares-1.15.0        |  100 KB | : 100% 1.0/1 [00:00<00:00, 13.85it/s]
cffi-1.12.3          |  218 KB | : 100% 1.0/1 [00:00<00:00, 11.59it/s]
fastavro-0.22.3      |  408 KB | : 100% 1.0/1 [00:00<00:00,  8.97it/s]
pillow-6.1.0         |  634 KB | : 100% 1.0/1 [00:00<00:00,  5.53it/s]               
pytz-2019.2          |  228 KB | : 100% 1.0/1 [00:00<00:00,  4.80it/s]              
thrift-cpp-0.12.0    |  2.4 MB | : 100% 1.0/1 [00:00<00:00,  2.11it/s]               
grpc-cpp-1.23.0      |  4.5 MB | : 100% 1.0/1 [00:00<00:00,  1.08it/s]               
requests-oauthlib-1. |   19 KB | : 100% 1.0/1 [00:00<00:00, 25.90it/s]
readline-8.0         |  441 KB | : 100% 1.0/1 [00:00<00:00,  8.89it/s]
requests-2.22.0      |   84 KB | : 100% 1.0/1 [00:00<00:00, 15.26it/s]
libcudf-0.9.0        | 29.1 MB | : 100% 1.0/1 [00:05<00:00,  5.77s/it]               
bzip2-1.0.8          |  396 KB | : 100% 1.0/1 [00:00<00:00,  9.12it/s]
openssl-1.1.1c       |  2.1 MB | : 100% 1.0/1 [00:00<00:00,  3.03it/s]              
boost-cpp-1.70.0     | 21.1 MB | : 100% 1.0/1 [00:08<00:00,  8.32s/it]               
pynvml-8.0.2         |   30 KB | : 100% 1.0/1 [00:00<00:00, 19.83it/s]
libnvstrings-0.9.0   | 16.8 MB | : 100% 1.0/1 [00:05<00:00,  5.25s/it]               
lz4-c-1.8.3          |  187 KB | : 100% 1.0/1 [00:00<00:00, 13.98it/s]
tornado-6.0.3        |  636 KB | : 100% 1.0/1 [00:00<00:00,  4.81it/s]               
distributed-2.3.2    |  370 KB | : 100% 1.0/1 [00:00<00:00,  5.68it/s]               
pyarrow-0.14.1       |  2.8 MB | : 100% 1.0/1 [00:00<00:00,  1.22it/s]               
arrow-cpp-0.14.1     | 17.3 MB | : 100% 1.0/1 [00:02<00:00,  2.78s/it]              
numpy-1.17.0         |  5.2 MB | : 100% 1.0/1 [00:01<00:00,  1.16s/it]               
yaml-0.1.7           |   78 KB | : 100% 1.0/1 [00:00<00:00, 20.83it/s]
sortedcontainers-2.1 |   25 KB | : 100% 1.0/1 [00:00<00:00, 26.64it/s]
zstd-1.4.0           |  928 KB | : 100% 1.0/1 [00:00<00:00,  5.60it/s]               
dlpack-0.2           |   12 KB | : 100% 1.0/1 [00:00<00:00, 29.11it/s]
urllib3-1.25.3       |  187 KB | : 100% 1.0/1 [00:00<00:00,  9.68it/s]
cudf-0.9.0           |  4.8 MB | : 100% 1.0/1 [00:01<00:00,  1.47s/it]               
py-xgboost-0.90.rapi |   87 KB | : 100% 1.0/1 [00:00<00:00,  5.01it/s]                
pyjwt-1.7.1          |   17 KB | : 100% 1.0/1 [00:00<00:00, 21.50it/s]
locket-0.2.0         |    6 KB | : 100% 1.0/1 [00:00<00:00, 19.46it/s]
wheel-0.33.6         |   35 KB | : 100% 1.0/1 [00:00<00:00, 21.14it/s]
dask-2.3.0           |    4 KB | : 100% 1.0/1 [00:00<00:00, 29.72it/s]
pyopenssl-19.0.0     |   81 KB | : 100% 1.0/1 [00:00<00:00, 18.50it/s]
numba-0.45.1         |  3.1 MB | : 100% 1.0/1 [00:00<00:00,  1.09it/s]               
brotli-1.0.7         |  1.0 MB | : 100% 1.0/1 [00:00<00:00,  5.73it/s]               
snappy-1.1.7         |   39 KB | : 100% 1.0/1 [00:00<00:00, 22.96it/s]
scikit-learn-0.21.3  |  6.7 MB | : 100% 1.0/1 [00:01<00:00,  1.39s/it]               
freetype-2.10.0      |  884 KB | : 100% 1.0/1 [00:00<00:00,  4.60it/s]               
pyyaml-5.1.2         |  184 KB | : 100% 1.0/1 [00:00<00:00, 13.34it/s]
libcuml-0.9.1        | 29.7 MB | : 100% 1.0/1 [00:06<00:00,  6.52s/it]               
libffi-3.2.1         |   46 KB | : 100% 1.0/1 [00:00<00:00, 19.42it/s]
libcugraph-0.9.0     | 11.3 MB | : 100% 1.0/1 [00:02<00:00,  2.31s/it]               
double-conversion-3. |   85 KB | : 100% 1.0/1 [00:00<00:00, 20.03it/s]
llvmlite-0.29.0      | 19.9 MB | : 100% 1.0/1 [00:03<00:00,  3.05s/it]               
parquet-cpp-1.5.1    |    3 KB | : 100% 1.0/1 [00:00<00:00, 26.03it/s]
msgpack-python-0.6.1 |   89 KB | : 100% 1.0/1 [00:00<00:00, 17.11it/s]
icu-64.2             | 12.6 MB | : 100% 1.0/1 [00:01<00:00,  1.78s/it]               
jinja2-2.10.1        |   91 KB | : 100% 1.0/1 [00:00<00:00, 17.11it/s]
google-auth-1.6.3    |   45 KB | : 100% 1.0/1 [00:00<00:00, 18.48it/s]
libpng-1.6.37        |  343 KB | : 100% 1.0/1 [00:00<00:00, 11.15it/s]
six-1.12.0           |   22 KB | : 100% 1.0/1 [00:00<00:00, 25.52it/s]
oauthlib-3.0.1       |   82 KB | : 100% 1.0/1 [00:00<00:00, 14.43it/s]
tk-8.6.9             |  3.2 MB | : 100% 1.0/1 [00:00<00:00,  1.58it/s]               
libcumlprims-0.9.0   |  3.9 MB | : 100% 1.0/1 [00:01<00:00,  1.56s/it]               
pyasn1-0.4.6         |   52 KB | : 100% 1.0/1 [00:00<00:00, 17.38it/s]
cryptography-2.7     |  607 KB | : 100% 1.0/1 [00:00<00:00,  4.86it/s]               
pycparser-2.19       |  173 KB | : 100% 1.0/1 [00:00<00:00, 12.65it/s]
packaging-19.0       |   23 KB | : 100% 1.0/1 [00:00<00:00, 24.10it/s]
cugraph-0.9.0        |  1.3 MB | : 100% 1.0/1 [00:00<00:00,  2.56it/s]               
libtiff-4.0.10       |  587 KB | : 100% 1.0/1 [00:00<00:00,  7.67it/s]               
libstdcxx-ng-9.1.0   |  4.0 MB | : 100% 1.0/1 [00:00<00:00,  1.69it/s]               
dask-cudf-0.9.0      |   63 KB | : 100% 1.0/1 [00:00<00:00,  1.75it/s]                
libopenblas-0.3.7    |  7.6 MB | : 100% 1.0/1 [00:01<00:00,  1.27s/it]             
bokeh-1.3.4          |  4.0 MB | : 100% 1.0/1 [00:01<00:00,  1.34s/it]               
cudatoolkit-10.0.130 | 380.0 MB | : 100% 1.0/1 [00:45<00:00, 45.04s/it]                
heapdict-1.0.0       |    7 KB | : 100% 1.0/1 [00:00<00:00, 19.05it/s]
pyparsing-2.4.2      |   57 KB | : 100% 1.0/1 [00:00<00:00, 18.17it/s]
cytoolz-0.10.0       |  429 KB | : 100% 1.0/1 [00:00<00:00,  7.47it/s]               
idna-2.8             |  132 KB | : 100% 1.0/1 [00:00<00:00, 16.54it/s]
ncurses-6.1          |  1.3 MB | : 100% 1.0/1 [00:00<00:00,  1.23it/s]              
markupsafe-1.1.1     |   26 KB | : 100% 1.0/1 [00:00<00:00, 20.88it/s]
cython-0.29.13       |  2.2 MB | : 100% 1.0/1 [00:00<00:00,  1.78it/s]               
google-auth-oauthlib |   18 KB | : 100% 1.0/1 [00:00<00:00, 21.45it/s]
gflags-2.2.2         |  177 KB | : 100% 1.0/1 [00:00<00:00,  6.69it/s]                
libevent-2.1.10      |  1.3 MB | : 100% 1.0/1 [00:00<00:00,  2.84it/s]               
libxgboost-0.90.rapi | 33.2 MB | : 100% 1.0/1 [00:07<00:00,  7.72s/it]               
pandas-0.24.2        | 11.1 MB | : 100% 1.0/1 [00:02<00:00,  2.25s/it]               
cloudpickle-1.2.1    |   22 KB | : 100% 1.0/1 [00:00<00:00, 21.78it/s]
libblas-3.8.0        |   10 KB | : 100% 1.0/1 [00:00<00:00, 25.26it/s]
python-3.6.7         | 34.6 MB | : 100% 1.0/1 [00:04<00:00,  4.64s/it]               
python-dateutil-2.8. |  219 KB | : 100% 1.0/1 [00:00<00:00, 14.63it/s]
joblib-0.13.2        |  180 KB | : 100% 1.0/1 [00:00<00:00, 10.47it/s]
blinker-1.4          |   13 KB | : 100% 1.0/1 [00:00<00:00, 22.48it/s]
partd-1.0.0          |   16 KB | : 100% 1.0/1 [00:00<00:00, 26.99it/s]
certifi-2019.6.16    |  149 KB | : 100% 1.0/1 [00:00<00:00, 20.28it/s]
zlib-1.2.11          |  105 KB | : 100% 1.0/1 [00:00<00:00, 19.25it/s]
tblib-1.4.0          |   12 KB | : 100% 1.0/1 [00:00<00:00, 28.22it/s]
fsspec-0.4.1         |   39 KB | : 100% 1.0/1 [00:00<00:00, 16.07it/s]
scipy-1.3.1          | 18.1 MB | : 100% 1.0/1 [00:03<00:00,  3.00s/it]               
decorator-4.4.0      |   11 KB | : 100% 1.0/1 [00:00<00:00, 25.91it/s]
cachetools-2.1.0     |   10 KB | : 100% 1.0/1 [00:00<00:00, 24.87it/s]
xz-5.2.4             |  366 KB | : 100% 1.0/1 [00:00<00:00,  9.22it/s]               
jpeg-9c              |  251 KB | : 100% 1.0/1 [00:00<00:00, 10.75it/s]
click-7.0            |   61 KB | : 100% 1.0/1 [00:00<00:00, 20.15it/s]
pysocks-1.7.0        |   26 KB | : 100% 1.0/1 [00:00<00:00, 20.41it/s]
liblapack-3.8.0      |   10 KB | : 100% 1.0/1 [00:00<00:00, 26.20it/s]
setuptools-41.2.0    |  634 KB | : 100% 1.0/1 [00:00<00:00,  4.46it/s]               
asn1crypto-0.24.0    |  154 KB | : 100% 1.0/1 [00:00<00:00, 11.07it/s]
glog-0.4.0           |  104 KB | : 100% 1.0/1 [00:00<00:00, 18.62it/s]
dask-core-2.3.0      |  574 KB | : 100% 1.0/1 [00:00<00:00,  4.83it/s]              
olefile-0.46         |   31 KB | : 100% 1.0/1 [00:00<00:00, 19.32it/s]
psutil-5.6.3         |  322 KB | : 100% 1.0/1 [00:00<00:00,  8.87it/s]
xgboost-0.90.rapidsd |   12 KB | : 100% 1.0/1 [00:00<00:00,  2.40it/s] 
libcblas-3.8.0       |   10 KB | : 100% 1.0/1 [00:00<00:00, 23.89it/s]
zict-1.0.0           |   10 KB | : 100% 1.0/1 [00:00<00:00, 22.89it/s]
_libgcc_mutex-0.1    |    3 KB | : 100% 1.0/1 [00:00<00:00, 31.51it/s]
rmm-0.9.0            |   14 KB | : 100% 1.0/1 [00:00<00:00,  5.63it/s] 
re2-2019.08.01       |  420 KB | : 100% 1.0/1 [00:00<00:00,  8.94it/s]
gcsfs-0.3.0          |   19 KB | : 100% 1.0/1 [00:00<00:00, 24.16it/s]
libgfortran-ng-7.3.0 |  1.3 MB | : 100% 1.0/1 [00:00<00:00,  3.77it/s]               
librmm-0.9.0         |   44 KB | : 100% 1.0/1 [00:00<00:00,  5.93it/s]               
chardet-3.0.4        |  190 KB | : 100% 1.0/1 [00:00<00:00, 11.50it/s]
pyasn1-modules-0.2.6 |   47 KB | : 100% 1.0/1 [00:00<00:00, 15.94it/s]
libgcc-ng-9.1.0      |  8.1 MB | : 100% 1.0/1 [00:01<00:00,  1.12s/it]              
uriparser-0.9.3      |   49 KB | : 100% 1.0/1 [00:00<00:00, 18.56it/s]
nvstrings-0.9.0      |  124 KB | : 100% 1.0/1 [00:00<00:00,  4.14it/s]                
libprotobuf-3.8.0    |  4.7 MB | : 100% 1.0/1 [00:00<00:00,  1.13it/s]               
rsa-3.4.2            |   31 KB | : 100% 1.0/1 [00:00<00:00, 20.01it/s]
ca-certificates-2019 |  145 KB | : 100% 1.0/1 [00:00<00:00, 16.25it/s]
nccl-2.4.6.1         | 66.6 MB | : 100% 1.0/1 [00:08<00:00,  8.80s/it]              
pip-19.2.2           |  1.9 MB | : 100% 1.0/1 [00:00<00:00,  1.86it/s]              
Copying shared object files to /usr/lib
Solving environment: done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - blazingsql-calcite
    - blazingsql-orchestrator
    - blazingsql-python
    - blazingsql-ral
    - cudatoolkit=10.0
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    xorg-renderproto-0.11.1    |    h14c3975_1002           8 KB  conda-forge
    libarchive-3.3.3           |    hb44662c_1005         1.4 MB  conda-forge
    rapidjson-1.1.0            |    he1b5a44_1002         104 KB  conda-forge
    openjdk-11.0.1             |    h46a85a0_1017       172.1 MB  conda-forge
    blazingsql-orchestrator-0.4.0|   py36hf484d3e_0         1.5 MB  felipeblazing/label/cuda10.0
    flatbuffers-1.11.0         |       he1b5a44_0         862 KB  conda-forge
    curl-7.65.3                |       hf8cf82a_0         132 KB  conda-forge
    maven-3.6.0                |                0         8.6 MB  conda-forge
    xorg-libxi-1.7.10          |       h516909a_0          45 KB  conda-forge
    giflib-5.1.7               |       h516909a_1         455 KB  conda-forge
    xorg-libxdmcp-1.1.3        |       h516909a_0          18 KB  conda-forge
    libgpg-error-1.36          |       he1b5a44_0         259 KB  conda-forge
    conda-package-handling-1.4.1|           py36_0         262 KB  conda-forge
    fontconfig-2.13.1          |    h86ecdb6_1001         340 KB  conda-forge
    blazingsql-communication-0.4.0|   py36hf484d3e_0         5.2 MB  felipeblazing/label/cuda10.0
    libuv-1.31.0               |       h516909a_0         878 KB  conda-forge
    xorg-recordproto-1.14.2    |    h14c3975_1002           7 KB  conda-forge
    python-libarchive-c-2.8    |        py36_1004          21 KB  conda-forge
    conda-4.7.11               |           py36_0         3.0 MB  conda-forge
    blazingsql-python-0.4.0    |   py36hf484d3e_0          43 KB  felipeblazing/label/cuda10.0
    pthread-stubs-0.4          |    h14c3975_1001           5 KB  conda-forge
    xorg-kbproto-1.0.7         |    h14c3975_1002          26 KB  conda-forge
    zeromq-4.3.2               |       he1b5a44_2         668 KB  conda-forge
    libgcrypt-1.8.4            |    hf484d3e_1000         614 KB  conda-forge
    libgsasl-1.8.0             |    h19a2143_1004         132 KB  conda-forge
    libxcb-1.13                |    h14c3975_1002         396 KB  conda-forge
    blazingsql-io-0.4.0        |   py36hf484d3e_0         253 KB  felipeblazing/label/cuda10.0
    gmock-1.8.1                |             1004         288 KB  conda-forge
    xorg-libxau-1.0.9          |       h14c3975_0          13 KB  conda-forge
    xorg-libxfixes-5.0.3       |    h516909a_1004          17 KB  conda-forge
    blazingsql-calcite-0.4.0   |   py36hf484d3e_0        33.6 MB  felipeblazing/label/cuda10.0
    krb5-1.16.3                |    h05b26f9_1001         1.4 MB  conda-forge
    xorg-libxtst-1.2.3         |    h14c3975_1002          30 KB  conda-forge
    xorg-libx11-1.6.8          |       h516909a_0         907 KB  conda-forge
    lcms2-2.9                  |       h2e4bb80_0         423 KB  conda-forge
    gtest-1.8.1                |    h6bb024c_1004         617 KB  conda-forge
    xorg-libxext-1.3.4         |       h516909a_0          51 KB  conda-forge
    xorg-libxrender-0.9.10     |    h516909a_1002          31 KB  conda-forge
    libiconv-1.15              |    h516909a_1005         2.0 MB  conda-forge
    libsodium-1.0.17           |       h516909a_0         330 KB  conda-forge
    lzo-2.10                   |    h14c3975_1000         319 KB  conda-forge
    xorg-xextproto-7.3.0       |    h14c3975_1002          27 KB  conda-forge
    cmake-3.15.2               |       hf94ab9c_0        11.5 MB  conda-forge
    libntlm-1.4                |    h14c3975_1002          32 KB  conda-forge
    blazingsql-protocol-0.4.0  |   py36hf484d3e_0         262 KB  felipeblazing/label/cuda10.0
    xorg-fixesproto-5.0        |    h14c3975_1002           8 KB  conda-forge
    rhash-1.3.6                |    h14c3975_1001         141 KB  conda-forge
    libhdfs3-2.3               |    h311b756_1006         8.9 MB  conda-forge
    expat-2.2.5                |    he1b5a44_1003         191 KB  conda-forge
    libssh2-1.8.2              |       h22169c7_2         257 KB  conda-forge
    libcurl-7.65.3             |       hda55be3_0         575 KB  conda-forge
    xorg-xproto-7.0.31         |    h14c3975_1007          72 KB  conda-forge
    gettext-0.19.8.1           |    hc5be6a0_1002         3.6 MB  conda-forge
    xorg-inputproto-2.3.2      |    h14c3975_1002          18 KB  conda-forge
    tqdm-4.34.0                |             py_0          41 KB  conda-forge
    boost-1.70.0               |   py36h9de70de_1         337 KB  conda-forge
    blazingsql-ral-0.4.0       |   py36hf484d3e_0         3.9 MB  felipeblazing/label/cuda10.0
    alsa-lib-1.1.5             |    h516909a_1001         547 KB  conda-forge
    libuuid-2.32.1             |    h14c3975_1000          26 KB  conda-forge
    libxml2-2.9.9              |       hee79883_2         1.3 MB  conda-forge
    ------------------------------------------------------------
                                           Total:       268.9 MB

The following NEW packages will be INSTALLED:

    alsa-lib:                 1.1.5-h516909a_1001    conda-forge                 
    blazingsql-calcite:       0.4.0-py36hf484d3e_0   felipeblazing/label/cuda10.0
    blazingsql-communication: 0.4.0-py36hf484d3e_0   felipeblazing/label/cuda10.0
    blazingsql-io:            0.4.0-py36hf484d3e_0   felipeblazing/label/cuda10.0
    blazingsql-orchestrator:  0.4.0-py36hf484d3e_0   felipeblazing/label/cuda10.0
    blazingsql-protocol:      0.4.0-py36hf484d3e_0   felipeblazing/label/cuda10.0
    blazingsql-python:        0.4.0-py36hf484d3e_0   felipeblazing/label/cuda10.0
    blazingsql-ral:           0.4.0-py36hf484d3e_0   felipeblazing/label/cuda10.0
    boost:                    1.70.0-py36h9de70de_1  conda-forge                 
    cmake:                    3.15.2-hf94ab9c_0      conda-forge                 
    conda-package-handling:   1.4.1-py36_0           conda-forge                 
    curl:                     7.65.3-hf8cf82a_0      conda-forge                 
    expat:                    2.2.5-he1b5a44_1003    conda-forge                 
    flatbuffers:              1.11.0-he1b5a44_0      conda-forge                 
    fontconfig:               2.13.1-h86ecdb6_1001   conda-forge                 
    gettext:                  0.19.8.1-hc5be6a0_1002 conda-forge                 
    giflib:                   5.1.7-h516909a_1       conda-forge                 
    gmock:                    1.8.1-1004             conda-forge                 
    gtest:                    1.8.1-h6bb024c_1004    conda-forge                 
    krb5:                     1.16.3-h05b26f9_1001   conda-forge                 
    lcms2:                    2.9-h2e4bb80_0         conda-forge                 
    libarchive:               3.3.3-hb44662c_1005    conda-forge                 
    libcurl:                  7.65.3-hda55be3_0      conda-forge                 
    libgcrypt:                1.8.4-hf484d3e_1000    conda-forge                 
    libgpg-error:             1.36-he1b5a44_0        conda-forge                 
    libgsasl:                 1.8.0-h19a2143_1004    conda-forge                 
    libhdfs3:                 2.3-h311b756_1006      conda-forge                 
    libiconv:                 1.15-h516909a_1005     conda-forge                 
    libntlm:                  1.4-h14c3975_1002      conda-forge                 
    libsodium:                1.0.17-h516909a_0      conda-forge                 
    libssh2:                  1.8.2-h22169c7_2       conda-forge                 
    libuuid:                  2.32.1-h14c3975_1000   conda-forge                 
    libuv:                    1.31.0-h516909a_0      conda-forge                 
    libxcb:                   1.13-h14c3975_1002     conda-forge                 
    libxml2:                  2.9.9-hee79883_2       conda-forge                 
    lzo:                      2.10-h14c3975_1000     conda-forge                 
    maven:                    3.6.0-0                conda-forge                 
    openjdk:                  11.0.1-h46a85a0_1017   conda-forge                 
    pthread-stubs:            0.4-h14c3975_1001      conda-forge                 
    python-libarchive-c:      2.8-py36_1004          conda-forge                 
    rapidjson:                1.1.0-he1b5a44_1002    conda-forge                 
    rhash:                    1.3.6-h14c3975_1001    conda-forge                 
    tqdm:                     4.34.0-py_0            conda-forge                 
    xorg-fixesproto:          5.0-h14c3975_1002      conda-forge                 
    xorg-inputproto:          2.3.2-h14c3975_1002    conda-forge                 
    xorg-kbproto:             1.0.7-h14c3975_1002    conda-forge                 
    xorg-libx11:              1.6.8-h516909a_0       conda-forge                 
    xorg-libxau:              1.0.9-h14c3975_0       conda-forge                 
    xorg-libxdmcp:            1.1.3-h516909a_0       conda-forge                 
    xorg-libxext:             1.3.4-h516909a_0       conda-forge                 
    xorg-libxfixes:           5.0.3-h516909a_1004    conda-forge                 
    xorg-libxi:               1.7.10-h516909a_0      conda-forge                 
    xorg-libxrender:          0.9.10-h516909a_1002   conda-forge                 
    xorg-libxtst:             1.2.3-h14c3975_1002    conda-forge                 
    xorg-recordproto:         1.14.2-h14c3975_1002   conda-forge                 
    xorg-renderproto:         0.11.1-h14c3975_1002   conda-forge                 
    xorg-xextproto:           7.3.0-h14c3975_1002    conda-forge                 
    xorg-xproto:              7.0.31-h14c3975_1007   conda-forge                 
    zeromq:                   4.3.2-he1b5a44_2       conda-forge                 

The following packages will be UPDATED:

    conda:                    4.5.4-py36_0                                        --> 4.7.11-py36_0 conda-forge


Downloading and Extracting Packages
xorg-renderproto-0.1 |    8 KB | : 100% 1.0/1 [00:00<00:00, 20.34it/s]
libarchive-3.3.3     |  1.4 MB | : 100% 1.0/1 [00:00<00:00,  2.10it/s]               
rapidjson-1.1.0      |  104 KB | : 100% 1.0/1 [00:00<00:00, 11.83it/s]
openjdk-11.0.1       | 172.1 MB | : 100% 1.0/1 [00:19<00:00, 19.24s/it]               
blazingsql-orchestra |  1.5 MB | : 100% 1.0/1 [00:00<00:00,  1.10it/s]              
flatbuffers-1.11.0   |  862 KB | : 100% 1.0/1 [00:00<00:00,  5.94it/s]               
curl-7.65.3          |  132 KB | : 100% 1.0/1 [00:00<00:00, 21.68it/s]
maven-3.6.0          |  8.6 MB | : 100% 1.0/1 [00:00<00:00,  1.15it/s]               
xorg-libxi-1.7.10    |   45 KB | : 100% 1.0/1 [00:00<00:00, 20.66it/s]
giflib-5.1.7         |  455 KB | : 100% 1.0/1 [00:00<00:00,  8.58it/s]
xorg-libxdmcp-1.1.3  |   18 KB | : 100% 1.0/1 [00:00<00:00, 25.28it/s]
libgpg-error-1.36    |  259 KB | : 100% 1.0/1 [00:00<00:00, 10.71it/s]
conda-package-handli |  262 KB | : 100% 1.0/1 [00:00<00:00, 13.28it/s]
fontconfig-2.13.1    |  340 KB | : 100% 1.0/1 [00:00<00:00,  9.37it/s]
blazingsql-communica |  5.2 MB | : 100% 1.0/1 [00:01<00:00,  1.20s/it]               
libuv-1.31.0         |  878 KB | : 100% 1.0/1 [00:01<00:00, 34.47s/it]               
xorg-recordproto-1.1 |    7 KB | : 100% 1.0/1 [00:00<00:00, 28.95it/s]
python-libarchive-c- |   21 KB | : 100% 1.0/1 [00:00<00:00, 24.77it/s]
conda-4.7.11         |  3.0 MB | : 100% 1.0/1 [00:00<00:00,  1.56it/s]               
blazingsql-python-0. |   43 KB | : 100% 1.0/1 [00:00<00:00,  3.76it/s]                
pthread-stubs-0.4    |    5 KB | : 100% 1.0/1 [00:00<00:00, 28.08it/s]
xorg-kbproto-1.0.7   |   26 KB | : 100% 1.0/1 [00:00<00:00, 21.60it/s]
zeromq-4.3.2         |  668 KB | : 100% 1.0/1 [00:00<00:00,  5.82it/s]               
libgcrypt-1.8.4      |  614 KB | : 100% 1.0/1 [00:00<00:00,  7.81it/s]               
libgsasl-1.8.0       |  132 KB | : 100% 1.0/1 [00:00<00:00, 14.79it/s]
libxcb-1.13          |  396 KB | : 100% 1.0/1 [00:00<00:00,  6.71it/s]              
blazingsql-io-0.4.0  |  253 KB | : 100% 1.0/1 [00:00<00:00,  4.15it/s]                
gmock-1.8.1          |  288 KB | : 100% 1.0/1 [00:00<00:00, 10.35it/s]
xorg-libxau-1.0.9    |   13 KB | : 100% 1.0/1 [00:00<00:00, 28.72it/s]
xorg-libxfixes-5.0.3 |   17 KB | : 100% 1.0/1 [00:00<00:00, 29.29it/s]
blazingsql-calcite-0 | 33.6 MB | : 100% 1.0/1 [00:03<00:00,  3.90s/it]               
krb5-1.16.3          |  1.4 MB | : 100% 1.0/1 [00:00<00:00,  3.51it/s]               
xorg-libxtst-1.2.3   |   30 KB | : 100% 1.0/1 [00:00<00:00, 25.95it/s]
xorg-libx11-1.6.8    |  907 KB | : 100% 1.0/1 [00:00<00:00,  4.99it/s]               
lcms2-2.9            |  423 KB | : 100% 1.0/1 [00:00<00:00,  9.38it/s]
gtest-1.8.1          |  617 KB | : 100% 1.0/1 [00:00<00:00,  6.21it/s]               
xorg-libxext-1.3.4   |   51 KB | : 100% 1.0/1 [00:00<00:00, 21.21it/s]
xorg-libxrender-0.9. |   31 KB | : 100% 1.0/1 [00:00<00:00, 23.27it/s]
libiconv-1.15        |  2.0 MB | : 100% 1.0/1 [00:00<00:00,  3.55it/s]               
libsodium-1.0.17     |  330 KB | : 100% 1.0/1 [00:00<00:00,  9.14it/s]              
lzo-2.10             |  319 KB | : 100% 1.0/1 [00:00<00:00, 10.34it/s]
xorg-xextproto-7.3.0 |   27 KB | : 100% 1.0/1 [00:00<00:00, 20.60it/s]
cmake-3.15.2         | 11.5 MB | : 100% 1.0/1 [00:02<00:00,  2.41s/it]              
libntlm-1.4          |   32 KB | : 100% 1.0/1 [00:00<00:00, 19.90it/s]
blazingsql-protocol- |  262 KB | : 100% 1.0/1 [00:00<00:00,  1.39it/s]               
xorg-fixesproto-5.0  |    8 KB | : 100% 1.0/1 [00:00<00:00, 28.50it/s]
rhash-1.3.6          |  141 KB | : 100% 1.0/1 [00:00<00:00, 20.03it/s]
libhdfs3-2.3         |  8.9 MB | : 100% 1.0/1 [00:01<00:00,  1.35s/it]               
expat-2.2.5          |  191 KB | : 100% 1.0/1 [00:00<00:00, 12.87it/s]
libssh2-1.8.2        |  257 KB | : 100% 1.0/1 [00:00<00:00,  9.20it/s]               
libcurl-7.65.3       |  575 KB | : 100% 1.0/1 [00:00<00:00,  8.20it/s]
xorg-xproto-7.0.31   |   72 KB | : 100% 1.0/1 [00:00<00:00, 20.32it/s]
gettext-0.19.8.1     |  3.6 MB | : 100% 1.0/1 [00:01<00:00,  1.03s/it]               
xorg-inputproto-2.3. |   18 KB | : 100% 1.0/1 [00:00<00:00, 23.85it/s]
tqdm-4.34.0          |   41 KB | : 100% 1.0/1 [00:00<00:00, 19.45it/s]
boost-1.70.0         |  337 KB | : 100% 1.0/1 [00:00<00:00,  6.88it/s]               
blazingsql-ral-0.4.0 |  3.9 MB | : 100% 1.0/1 [00:01<00:00,  1.14s/it]               
alsa-lib-1.1.5       |  547 KB | : 100% 1.0/1 [00:00<00:00,  5.80it/s]               
libuuid-2.32.1       |   26 KB | : 100% 1.0/1 [00:00<00:00, 23.87it/s]
libxml2-2.9.9        |  1.3 MB | : 100% 1.0/1 [00:00<00:00,  2.38it/s]               
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Collecting flatbuffers
  Downloading https://files.pythonhosted.org/packages/c9/84/adf5837f96c39990bc55afdfddf460b38b4562f50341359afa32e4a98de7/flatbuffers-1.11-py2.py3-none-any.whl
Installing collected packages: flatbuffers
Successfully installed flatbuffers-1.11

*********************************************
Your Google Colab instance is RAPIDS ready!
*********************************************

*********************************************
Your Google Colab instance is BlazingSQL ready!
*********************************************
CPU times: user 2.1 s, sys: 266 ms, total: 2.36 s
Wall time: 6min 43s

SyntaxError: ignored

In [14]:
%%time
!bash bsql-colab.sh

File ‘env-check.py’ already there; not retrieving.

Checking for GPU type:
*********************************************
Woo! Your instance has the right kind of GPU!
*********************************************


*********************************************
Your Google Colab instance is RAPIDS ready!
*********************************************

*********************************************
Your Google Colab instance is BlazingSQL ready!
*********************************************
CPU times: user 16.9 ms, sys: 11.4 ms, total: 28.2 ms
Wall time: 1.97 s


In [0]:
import sys, os

sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

#we are working on removing the need to call these functions and just initializing them in BlazingContext
import subprocess
subprocess.Popen(['blazingsql-orchestrator', '9100', '8889', '127.0.0.1', '8890'],stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
subprocess.Popen(['java', '-jar', '/usr/local/lib/blazingsql-algebra.jar', '-p', '8890'])
import pyblazing.apiv2.context as cont
cont.runRal()

In [17]:
# Import RAPIDS AI stack
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

Already connected to the Orchestrator


In [18]:
!wget https://blazingsql-colab.s3.amazonaws.com/netflow_data/nf-chunk2.csv

--2019-08-27 12:59:56--  https://blazingsql-colab.s3.amazonaws.com/netflow_data/nf-chunk2.csv
Resolving blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)... 52.216.178.59
Connecting to blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)|52.216.178.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2725056295 (2.5G) [text/csv]
Saving to: ‘nf-chunk2.csv’

nf-chunk2.csv       100%[===================>]   2.54G  19.8MB/s    in 2m 22s  

2019-08-27 13:02:18 (18.3 MB/s) - ‘nf-chunk2.csv’ saved [2725056295/2725056295]



In [19]:
%%time
#Load CSVs into GPU DataFrames (gdf)
netflow_gdf = cudf.read_csv('nf-chunk2.csv')

CPU times: user 3.84 s, sys: 1.25 s, total: 5.1 s
Wall time: 5.09 s


In [20]:
%%time
#Create BlazingSQL Tables - There is no copy in this process
bc.create_table('netflow', netflow_gdf)

CPU times: user 26.8 ms, sys: 0 ns, total: 26.8 ms
Wall time: 79.8 ms


In [21]:
%%time
sql = '''
SELECT
  a.firstSeenSrcIp as source,
  a.firstSeenDestIp as destination,
  count(a.firstSeenDestPort) as targetPorts,
  SUM(a.firstSeenSrcTotalBytes) as bytesOut,
  SUM(a.firstSeenDestTotalBytes) as bytesIn,
  SUM(a.durationSeconds) as durationSeconds,
  MIN(parsedDate) as firstFlowDate,
  MAX(parsedDate) as lastFlowDate,
  COUNT(*) as attemptCount
  FROM
  main.netflow a
  GROUP BY
  a.firstSeenSrcIp,
  a.firstSeenDestIp
  '''

result = bc.sql(sql,['netflow']).get()

# print(result.columns)

result_gdf = result.columns
edges_df = result_gdf.to_pandas()
print(edges_df.head(10))

NOTE: You no longer need to send a table list to the .sql() funtion
  firstSeenSrcIp firstSeenDestIp  ...         lastFlowDate  attemptCount
0    172.30.1.10       10.0.0.12  ...  2013-04-03 12:11:40            69
1    172.30.1.56       172.0.0.1  ...  2013-04-03 22:05:39            25
2   172.10.1.234        10.0.0.5  ...  2013-04-03 15:11:07           104
3    172.30.2.60        10.0.0.9  ...  2013-04-03 12:12:37            82
4   172.30.2.125        10.0.0.9  ...  2013-04-03 12:12:37            69
5   172.10.1.106    10.199.250.2  ...  2013-04-03 10:12:35            40
6   172.30.1.201       172.0.0.1  ...  2013-04-03 23:06:00            29
7      10.1.0.76     172.10.1.82  ...  2013-04-03 09:55:05             1
8    172.10.1.89        10.0.0.5  ...  2013-04-03 15:17:39           112
9   172.10.1.162       10.0.0.11  ...  2013-04-03 14:58:35            87

[10 rows x 9 columns]
CPU times: user 59.9 ms, sys: 490 µs, total: 60.4 ms
Wall time: 1.45 s


In [22]:
%%time
# Install Spark 
# Note: This installs Spark (version 2.4.1, as tested in April 2019)

!pip install pyspark

     |████████████████████████████████| 215.6MB 142kB/s 
     |████████████████████████████████| 204kB 58.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.3-py2.py3-none-any.whl size=215964963 sha256=6d4618811e2bd9ee0478800adc53ff456b913f65532694d7b9da3994f10a5d31
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark
CPU times: user 1.41 s, sys: 344 ms, total: 1.75 s
Wall time: 35.7 s


In [23]:
%%time
#I copied this cell's snippet from another Google Colab by Luca Canali here: https://colab.research.google.com/github/LucaCanali/sparkMeasure/blob/master/examples/SparkMeasure_Jupyter_Colab_Example.ipynb

from pyspark.sql import SparkSession

# Create Spark Session
# This example uses a local cluster, you can modify master to use  YARN or K8S if available 
# This example downloads sparkMeasure 0.13 for scala 2_11 from maven central

spark = SparkSession \
 .builder \
 .master("local[*]") \
 .appName("PySpark Netflow Benchmark code") \
 .config("spark.jars.packages","ch.cern.sparkmeasure:spark-measure_2.11:0.13")  \
 .getOrCreate()

CPU times: user 61.9 ms, sys: 27.6 ms, total: 89.5 ms
Wall time: 17.8 s


In [24]:
%%time

netflow_df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('nf-chunk2.csv')

CPU times: user 64.4 ms, sys: 107 ms, total: 171 ms
Wall time: 5min 10s


In [25]:
%%time
netflow_df.createOrReplaceTempView('netflow')

CPU times: user 139 µs, sys: 1.05 ms, total: 1.19 ms
Wall time: 182 ms


In [26]:
%%time
sql = '''
SELECT
  a.firstSeenSrcIp as source,
  a.firstSeenDestIp as destination,
  count(a.firstSeenDestPort) as targetPorts,
  SUM(a.firstSeenSrcTotalBytes) as bytesOut,
  SUM(a.firstSeenDestTotalBytes) as bytesIn,
  SUM(a.durationSeconds) as durationSeconds,
  MIN(parsedDate) as firstFlowDate,
  MAX(parsedDate) as lastFlowDate,
  COUNT(*) as attemptCount
  FROM
  netflow a
  GROUP BY
  a.firstSeenSrcIp,
  a.firstSeenDestIp
  '''

edges_df = spark.sql(sql)

edges_df.show()

+------------+---------------+-----------+--------+-------+---------------+-------------------+-------------------+------------+
|      source|    destination|targetPorts|bytesOut|bytesIn|durationSeconds|      firstFlowDate|       lastFlowDate|attemptCount|
+------------+---------------+-----------+--------+-------+---------------+-------------------+-------------------+------------+
| 172.10.1.13|239.255.255.250|         15|    2975|      0|              6|2013-04-03 06:36:19|2013-04-03 06:36:27|          15|
|172.30.1.204|239.255.255.250|          8|    1750|      0|              6|2013-04-03 06:36:13|2013-04-03 06:36:20|           8|
| 172.30.2.86|      172.0.0.1|          1|     540|      0|              2|2013-04-03 06:36:09|2013-04-03 06:36:09|           1|
|172.30.1.246|      172.0.0.1|         29|    2610|   2610|              0|2013-04-03 00:26:46|2013-04-03 23:06:00|          29|
| 172.30.1.51|239.255.255.250|         16|    3850|      0|             18|2013-04-03 06:35:22|20